In [63]:
from pyscf import gto, scf, mp, cc

a = 2 # bond length in a cluster
d = 10 # distance between each cluster
unit = 'b' # unit of length
na = 2  # size of a cluster (monomer)
nc = 4 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = '6-31g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

# atoms = '''
# O  -1.551007  -0.114520   0.000000
# H  -1.934259   0.762503   0.000000
# H  -0.599677   0.040712   0.000000
# O   1.350625   0.111469   0.000000
# H   1.680398  -0.373741  -0.758561
# H   1.680398  -0.373741   0.758561
# '''
# basis = 'ccpvdz'

mol = gto.M(atom=atoms, basis=basis, spin=0, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Thu Dec 18 19:00:55 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 8
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000

In [64]:
nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]


******** <class 'pyscf.cc.dfuccsd.UCCSD'> ********
CC2 = 0
CCSD nocc = (np.int64(4), np.int64(4)), nmo = (16, 16)
frozen orbitals 0
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-06
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 16000 MB (current use 1202 MB)
Init t2, MP2 energy = -0.00261200424110226
Init E_corr(UCCSD) = -0.00261200424110235
cycle = 1  E_corr(UCCSD) = -0.00373795786321811  dE = -0.00112595362  norm(t1,t2) = 0.0267405
cycle = 2  E_corr(UCCSD) = -0.00458290776085822  dE = -0.000844949898  norm(t1,t2) = 0.0251705
cycle = 3  E_corr(UCCSD) = -0.00732441227119782  dE = -0.00274150451  norm(t1,t2) = 0.0244075
cycle = 4  E_corr(UCCSD) = -0.0162151304261286  dE = -0.00889071815  norm(t1,t2) = 0.0231491
cycle = 5  E_corr(UCCSD) = -0.0852876673174359  dE = -0.0690725369  norm(t1,t2) = 0.0206339
cycle = 6  E_corr(UCCSD) = -0.0649337307412944  dE = 0.0203539366  norm(t1,t2) = 0.0624434
cycle = 7  E_corr(UCCSD) = -0.0482359853469157  

np.float64(-0.053532505507581506)

In [65]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob
print(frag_lolist)



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 3.8122236136182  delta_f= 3.81222  |g|= 0.0131022  1 KF 1 Hx
macro= 2  f(x)= 3.8122236136182  delta_f= 8.88178e-16  |g|= 5.97934e-08  1 KF 1 Hx
macro X = 2  f(x)= 3.8122236136182  |g|= 5.97934e-08  4 intor 2 KF 2 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin

In [66]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd
from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 1e-6
run_frg_list = [0]
chol_cut = 1e-5

mlno = ulnoccsd.ULNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
# frag_res = [None] * nfrag
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)

print(lno_frozen[0])
print(lno_frozen[1])

[9.999999999999999e-06, 1e-06]
LO occ proj: 1 active | 0 standby | 3 orthogonal
LO occ proj: 0 active | 0 standby | 4 orthogonal
[ 0  1  2  7  8  9 10 11 12 13 14 15]
[ 0  1  2  7  8  9 10 11 12 13 14 15]


In [67]:
(emp2,eccsd,ept), frag_msg = mlno.impurity_solve(mf, lno_coeff, uocc_loc, eris, frozen=lno_frozen)
print(emp2,eccsd,ept)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

-0.00032649379951794476 -0.006691563594637468 0.0


In [68]:
mol = mf.mol
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in lno_frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in lno_frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in lno_frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in lno_frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)

ncas = (nactorb_a, nactorb_b)
ncore = (nfrzocc_a, nfrzocc_b)
nelec = (nactocc_a, nactocc_b)
print(ncas)
print(ncore)
print(nelec)

(4, 4)
(3, 3)
(1, 1)


In [69]:
mo_occ = mlno.mo_occ
# mlno.verbose_imp = 4
lno_frozen, maskact = ulnoccsd.get_maskact(lno_frozen, [mo_occ[0].size, mo_occ[1].size])
mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=4)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

(emp2,eccsd,ept,ecct1_0,ecct2_0), t1, t2 =\
    ulno_afqmc.ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
prjlo = [uocc_loc[0].T.conj(), uocc_loc[1].T.conj()]

print(emp2,eccsd,ept,ecct1_0,ecct2_0)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

Init t2, MP2 energy = -0.000652987599035887

******** <class 'pyscf.lno.ulnoccsd.MODIFIED_UCCSD'> ********
CC2 = 0
CCSD nocc = (np.int64(1), np.int64(1)), nmo = (4, 4)
frozen orbitals [array([ 0,  1,  2,  7,  8,  9, 10, 11, 12, 13, 14, 15]), array([ 0,  1,  2,  7,  8,  9, 10, 11, 12, 13, 14, 15])]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-06
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 16000 MB (current use 1203 MB)
Init E_corr(MODIFIED_UCCSD) = -0.000652987599035887
cycle = 1  E_corr(MODIFIED_UCCSD) = -0.000934471324935842  dE = -0.000281483726  norm(t1,t2) = 0.0133701
cycle = 2  E_corr(MODIFIED_UCCSD) = -0.00114570617170959  dE = -0.000211234847  norm(t1,t2) = 0.0125852
cycle = 3  E_corr(MODIFIED_UCCSD) = -0.00183107650815864  dE = -0.000685370336  norm(t1

In [ ]:
# from jax import numpy as jnp
# pa = jnp.einsum('li,lk->ik',prja, prja)
# pb = jnp.einsum('li,lk->ik',prjb, prjb)
# print(pa)
# print(pb)

[[1.]]
[[0. 0.]
 [0. 0.]]


In [70]:
pa = uocc_loc[0] @ uocc_loc[0].T.conj()
pb = uocc_loc[1] @ uocc_loc[1].T.conj()
prjlo = [pa,pb]
print(pa)
print(pb)

[[1.]]
[[0.]]


In [70]:
options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 1,
        'seed': 98,
        'walker_type': 'uhf',
        'trial': 'uccsd_pt2_ad',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }

from mpi4py import MPI
if not MPI.Is_finalized():
    MPI.Finalize()

t1 = [t1[0]*0,t1[1]*0]
nelec, ncas = ulno_afqmc.prep_afqmc(
    mf,lno_coeff,t1,t2,lno_frozen,prjlo,
    options,chol_cut=chol_cut)

print(nelec, ncas)

NameError: name 't1' is not defined

In [1]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
from ad_afqmc.lno_afqmc import ulno_afqmc
ham_data, prop, trial, wave_data, sampler, options, _ = (
    ulno_afqmc._prep_afqmc())

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Using GPU.
# System: Linux
# Node Name: sharmagroup-rn
# Release: 6.14.0-37-generic
# Version: #37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2
# Machine: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: (10, 9)
# nelec: (4, 4)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 5
# n_blocks: 10
# n_walkers: 50
# seed: 24
# walker_type: uhf
# trial: uccsd_pt2
# dt: 0.005
# free_projection: False
# use_gpu: True
# max_error: 0.0005
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#


In [2]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])
# converged SCF energy = -10.8249478795192

-6.196168231681522


In [58]:
from jax import jvp, lax
def _eorb_pt2_ad(trial, walker_up, walker_dn, ham_data, wave_data):
    '''
    <HF|t2_i (h1mod+h2mod)|walker>/<HF|walker>
    note h1mod_pq = h1_pq - 1/2 v_prrq
    '''
    nocca, noccb = trial.nelec
    norba, norbb = trial.norb
    h1_mod = ham_data['h1_mod']
    chola, cholb = ham_data["chol"]
    chola = chola.reshape(-1, norba, norba)
    cholb = cholb.reshape(-1, norbb, norbb)
    chol = [chola, cholb]

    o0 = jnp.linalg.det(walker_up[:nocca,:]) * jnp.linalg.det(walker_dn[:noccb,:])

    # one body
    f1 = lambda a: trial._t2_exp1_orb(a,h1_mod,walker_up,walker_dn,wave_data)
    t2olp, d_overlap = jvp(f1, [0.0], [1.0])

    # two body
    def scanned_fun(carry, c):
        walker_up, walker_dn, wave_data = carry
        return carry, trial._d2_exp2_orb_i(c,walker_up,walker_dn,wave_data)

    _, d2_olp2_i = lax.scan(scanned_fun, (walker_up,walker_dn,wave_data), chol)
    d2_overlap = jnp.sum(d2_olp2_i)/2

    e1mod = d_overlap / o0
    e2mod = d2_overlap / o0
    t2eorb = e1mod + e2mod # <hf|t2_i(h1+h2)|walker>/<hf|walker>
    t2orb = t2olp / o0 # <t2_i>

    return jnp.real(t2eorb), jnp.real(t2orb)

In [59]:
import opt_einsum as oe 
def _eorb_pt2_tc(
    trial,
    walker_up,
    walker_dn,
    ham_data,
    wave_data):
    
    norb_a, norb_b = trial.norb
    nocc_a, nocc_b = trial.nelec
    # h0 = ham_data["h0"], ham_data["E0"]
    h1a, h1b = ham_data["h1"]
    t2aa, t2ab = wave_data["t2aa"], wave_data["t2ab"]
    t2ba, t2bb = wave_data["t2ba"], wave_data["t2bb"]
    chol_a, chol_b = ham_data["chol"]
    chol_a = chol_a.reshape(-1, norb_a, norb_a)
    chol_b = chol_b.reshape(-1, norb_b, norb_b)
    rot_chol_a = chol_a[:, :nocc_a, :]
    rot_chol_b = chol_b[:, :nocc_b, :]

    green_a = (walker_up.dot(jnp.linalg.inv(walker_up[:nocc_a, :]))).T # G_ip
    green_b = (walker_dn.dot(jnp.linalg.inv(walker_dn[:nocc_b, :]))).T
    green_occ_a = green_a[:, nocc_a:]
    green_occ_b = green_b[:, nocc_b:]
    greenp_a = jnp.vstack((green_occ_a, -jnp.eye(norb_a - nocc_a)))
    greenp_b = jnp.vstack((green_occ_b, -jnp.eye(norb_b - nocc_b)))

    # 1 body energy    
    hg_a = oe.contract("pj,pj->", h1a[:nocc_a, :], green_a, backend="jax")
    hg_b = oe.contract("pj,pj->", h1b[:nocc_b, :], green_b, backend="jax")
    e1_0 = hg_a + hg_b #  <HF|h1|walker>/<HF|walker>

    # double excitations
    # ij does not have anti-sym in LNO!!!
    t2g_aa_a_c = oe.contract("iajb,ia->jb", t2aa, green_occ_a, backend="jax") / 4
    t2g_aa_a_e = oe.contract("iajb,ja->ib", t2aa, green_occ_a, backend="jax") / 4
    t2g_bb_b_c = oe.contract("iajb,ia->jb", t2bb, green_occ_b, backend="jax") / 4
    t2g_bb_b_e = oe.contract("iajb,ja->ib", t2bb, green_occ_b, backend="jax") / 4
    t2g_ab_a = oe.contract("iajb,ia->jb", t2ab, green_occ_a, backend="jax") / 2
    t2g_ab_b = oe.contract("iajb,jb->ia", t2ab, green_occ_b, backend="jax") / 2
    t2g_ba_a = oe.contract("iajb,jb->ia", t2ba, green_occ_a, backend="jax") / 2
    t2g_ba_b = oe.contract("iajb,ia->jb", t2ba, green_occ_b, backend="jax") / 2
    gt2g_aa = oe.contract("jb,jb->", t2g_aa_a_c, green_occ_a, backend="jax")
    gt2g_bb = oe.contract("jb,jb->", t2g_bb_b_c, green_occ_b, backend="jax")
    gt2g_ab = oe.contract("jb,jb->", t2g_ab_a, green_occ_b, backend="jax")
    gt2g_ba = oe.contract("jb,jb->", t2g_ba_b, green_occ_a, backend="jax")
    gt2g =  (gt2g_aa + gt2g_bb) * 2 + (gt2g_ab + gt2g_ba)
    e1_2_1 = gt2g * e1_0

    # t_iajb G_ia G_jq Gp_pb
    t2_green_aaa_c = oe.contract('pb,jb,jq->pq', greenp_a, t2g_aa_a_c, green_a, backend="jax") # t_iajb G_ia G_jq Gp_pb (-)
    t2_green_aaa_e = oe.contract('pb,ib,iq->pq', greenp_a, t2g_aa_a_e, green_a, backend="jax") # t_iajb G_ja G_iq Gp_pb (+)
    t2_green_bbb_c = oe.contract('pb,jb,jq->pq', greenp_b, t2g_bb_b_c, green_b, backend="jax")
    t2_green_bbb_e = oe.contract('pb,ib,iq->pq', greenp_b, t2g_bb_b_e, green_b, backend="jax")
    t2_green_aba = oe.contract('pa,ia,iq->pq', greenp_a, t2g_ab_b, green_a, backend="jax")
    t2_green_baa = oe.contract('pb,jb,jq->pq', greenp_a, t2g_ba_b, green_a, backend="jax")
    t2_green_bab = oe.contract('pa,ia,iq->pq', greenp_b, t2g_ba_a, green_b, backend="jax")
    t2_green_abb = oe.contract('pb,jb,jq->pq', greenp_b, t2g_ab_a, green_b, backend="jax")
    t2_green_aaa = 2 * (t2_green_aaa_c - t2_green_aaa_e) # use the anti-sym of a <-> b
    t2_green_bbb = 2 * (t2_green_bbb_c - t2_green_bbb_e)
    e1_2_2_a = -oe.contract("pq,pq->", t2_green_aaa + t2_green_aba + t2_green_baa, h1a, backend="jax")
    e1_2_2_b = -oe.contract("pq,pq->", t2_green_bbb + t2_green_bab + t2_green_abb, h1b, backend="jax")
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2 # <HF|T2 h1|walker>/<HF|walker>

    # two body energy
    lg_a = oe.contract("gpj,qj->gpq", rot_chol_a, green_a, backend="jax")
    lg_b = oe.contract("gpj,qj->gpq", rot_chol_b, green_b, backend="jax")
    tr_lg_a = oe.contract("gpp->g", lg_a, backend="jax")
    tr_lg_b = oe.contract("gpp->g", lg_b, backend="jax")
    lg_0 = tr_lg_a + tr_lg_b
    e2_0_1 = oe.contract('g,g->', lg_0, lg_0) / 2.0
    e2_0_2 = - (oe.contract("gpq,gqp->", lg_a, lg_a, backend="jax")
                + oe.contract("gpq,gqp->", lg_b, lg_b, backend="jax")) / 2.0
    e2_0 = e2_0_1 + e2_0_2 # <HF|h2|walker>/<HF|walker>


    # double excitations
    e2_2_1 = e2_0 * gt2g
    lt2g_a = oe.contract("gpq,pq->g", chol_a, 2*t2_green_aaa + 2*(t2_green_aba + t2_green_baa),backend="jax")
    lt2g_b = oe.contract("gpq,pq->g", chol_b, 2*t2_green_bbb + 2*(t2_green_bab + t2_green_abb),backend="jax")
    e2_2_2_1 = -((lt2g_a + lt2g_b) @ lg_0) / 2.0

    def scanned_fun(carry, x):
        chol_a_i, rot_chol_a_i, chol_b_i, rot_chol_b_i = x
        gl_a_i = oe.contract("ir,pr->ip", green_a, chol_a_i, backend="jax")
        gl_b_i = oe.contract("ir,pr->ip", green_b, chol_b_i, backend="jax")
        lt2_green_a_i = oe.contract(
            "pi,ji->pj", rot_chol_a_i, 2*t2_green_aaa + 2*(t2_green_aba + t2_green_baa), backend="jax")
        lt2_green_b_i = oe.contract(
            "pi,ji->pj", rot_chol_b_i, 2*t2_green_bbb + 2*(t2_green_bab + t2_green_abb),backend="jax")
        carry[0] += (oe.contract("ip,ip->", gl_a_i, lt2_green_a_i, backend="jax")
                    + oe.contract("ip,ip->", gl_b_i, lt2_green_b_i, backend="jax")) / 2
        glgp_a_i = oe.contract("ip,pa->ia", gl_a_i, greenp_a, backend="jax")
        glgp_b_i = oe.contract("ip,pa->ia", gl_b_i, greenp_b, backend="jax")
        l2t2_aa = 0.5 * oe.contract("ia,jb,iajb->", glgp_a_i, glgp_a_i, t2aa, backend="jax")
        l2t2_ab = 0.5 * oe.contract("ia,jb,iajb->", glgp_a_i, glgp_b_i, t2ab, backend="jax")
        l2t2_ba = 0.5 * oe.contract("ia,jb,iajb->", glgp_b_i, glgp_a_i, t2ba, backend="jax")
        l2t2_bb = 0.5 * oe.contract("ia,jb,iajb->", glgp_b_i, glgp_b_i, t2bb, backend="jax")
        carry[1] += l2t2_aa + l2t2_ab + l2t2_ba + l2t2_bb
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(
        scanned_fun, [0.0, 0.0], (chol_a, rot_chol_a, chol_b, rot_chol_b)
    )
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <HF|T2 h2|walker>/<HF|walker>

    t2orb = gt2g # <HF|T1+T2|walker>/<HF|walker>
    e12bar = e1_0 + e2_0 # <HF|h1+h2|walker>/<HF|walker>
    t2eorb = e1_2 + e2_2 # <HF|T2(h1+h2)|walker>/<HF|walker>

    return jnp.real(t2eorb), jnp.real(t2orb), jnp.real(e12bar)

In [ ]:
# def _calc_eorb_bar(trial, walker_up, walker_dn, ham_data, wave_data):
#     '''
#     calculate the correlation energy of the Hamiltonian
#     transformed by exp(T1^dagger):
#     ecorr_bar = <psi_0|H_bar|walker_bar>/<psi_0|walker_bar>
#     |walker_bar> = exp(T1^dagger) |walker>
#     H_bar = exp(T1^dagger) H exp(-T1^dagger)
#     |psi_0> is the mean-field solution of H
#     '''
#     nocca, noccb = trial.nelec 
#     norba, norbb = trial.norb
#     prja, prjb = wave_data['prjlo']
#     e0 = ham_data['e0t1orb'] # <psi_0|H_bar|psi_0>
#     rot_focka = ham_data['fock_bar'][0][:nocca,:]
#     rot_fockb = ham_data['fock_bar'][1][:noccb,:]
#     rot_chola = ham_data['chol_bar'][0].reshape(-1,norba,norba)[:,:nocca,:]
#     rot_cholb = ham_data['chol_bar'][1].reshape(-1,norbb,norbb)[:,:noccb,:]

#     gfa = (walker_up.dot(jnp.linalg.inv(walker_up[:nocca, :]))).T
#     gfb = (walker_dn.dot(jnp.linalg.inv(walker_dn[:noccb, :]))).T
#     e1a = oe.contract('ia,ia->',gfa[:nocca,nocca:],rot_focka[:nocca,nocca:], backend="jax")
#     e1b = oe.contract('ia,ia->',gfb[:noccb,noccb:],rot_fockb[:noccb,noccb:], backend="jax")
#     e1 = e1a + e1b
    
#     lga = oe.contract('gia,ka->gik', rot_chola[:,:nocca,nocca:], gfa[:nocca,nocca:], backend="jax")
#     lgb = oe.contract('gia,ka->gik', rot_cholb[:,:noccb,noccb:], gfb[:noccb,noccb:], backend="jax")
#     e2aa = oe.contract('gik,ik,gjj->', lga, prja, lga, backend="jax") \
#         - oe.contract('gij,gjk,ik->',lga, lga, prja, backend="jax")
#     e2ab = oe.contract('gik,ik,gjj->', lga, prja, lgb, backend="jax")
#     e2ba = oe.contract('gik,ik,gjj->', lgb, prjb, lga, backend="jax")
#     e2bb = oe.contract('gik,ik,gjj->', lgb, prjb, lgb, backend="jax") \
#         - oe.contract('gij,gjk,ik->',lgb, lgb, prjb, backend="jax")
#     e2 = (e2aa + e2ab + e2ba + e2bb)/2
    
#     e_corr = e0 + e1 + e2
#     return e_corr

In [26]:
norba, norbb = trial.norb
nocca, noccb = trial.nelec
chola = ham_data["chol"][0].reshape(-1, norba, norba)
cholb = ham_data["chol"][1].reshape(-1, norbb, norbb)
v0_a = 0.5 * jnp.einsum("gpr,grq->pq", chola, chola, optimize="optimal")
v0_b = 0.5 * jnp.einsum("gpr,grq->pq", cholb, cholb, optimize="optimal")
h1mod_a = ham_data['h1'][0] - v0_a
h1mod_b = ham_data['h1'][1] - v0_b
ham_data['h1_mod'] = [h1mod_a,h1mod_b]

In [52]:
amp_file="amplitudes.npz"
amplitudes = np.load(amp_file)
t2aa = jnp.array(amplitudes["t2aa"])
t2ab = jnp.array(amplitudes["t2ab"])
t2bb = jnp.array(amplitudes["t2bb"])
wave_data["t2aa"] = t2aa
wave_data["t2ab"] = t2ab
wave_data["t2ba"] = t2ab.transpose(2,3,0,1)
wave_data["t2bb"] = t2bb

In [60]:
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
print(walker_up.shape,walker_dn.shape)

(10, 4) (9, 4)


In [62]:
walker_up_bar = wave_data['exp_t1a'] @ walker_up
walker_dn_bar = wave_data['exp_t1b'] @ walker_dn
te_ad, t_ad = _eorb_pt2_ad(trial,walker_up_bar,walker_dn_bar,ham_data,wave_data)
te_tc, t_tc, _ = _eorb_pt2_tc(trial,walker_up_bar,walker_dn_bar,ham_data,wave_data)
print(te_ad, t_ad)
print(te_tc, t_tc)

-0.009178823871565198 0.0
-0.009178823871565196 0.0


In [65]:
import numpy as np
walker_up_r = np.random.randn(*walker_up.shape)
walker_dn_r = np.random.randn(*walker_dn.shape)
# walker_up_r = wave_data['exp_t1a'] @ walker_up_r
# walker_dn_r = wave_data['exp_t1b'] @ walker_dn_r
te_ad, t_ad = _eorb_pt2_ad(trial,walker_up_r,walker_dn_r,ham_data,wave_data)
te_tc, t_tc, _ = _eorb_pt2_tc(trial,walker_up_r,walker_dn_r,ham_data,wave_data)
print(te_ad, t_ad)
print(te_tc, t_tc)

-1.6636454781411971 0.20925761409113638
-1.6636454781411971 0.20925761409113633


In [38]:
h1mod_ad, te_ad = _eorb_pt2_ad(trial,walker_up_r,walker_dn_r,ham_data,wave_data)
h1mod_tc, t2_tc = _eorb_pt2_tc(trial,walker_up_r,walker_dn_r,ham_data,wave_data)
print(h1mod_ad, te_ad)
print(h1mod_tc, t2_tc)

ValueError: too many values to unpack (expected 2)

In [24]:
import numpy as np
walker_up_r = np.random.randn(*walker_up.shape)
walker_dn_r = np.random.randn(*walker_dn.shape)
eorb_bar = _calc_eorb_bar(trial, walker_up_r, walker_dn_r, ham_data, wave_data)
print(jnp.real(eorb_bar))
eorb_0 = trial._calc_eorb(walker_up_r, walker_dn_r, ham_data, wave_data)
print(jnp.real(eorb_0))

-0.15113707712972277
-0.15113707973122825


In [10]:
# LNO-UCCSD (T1 = 0) fragment energy: -0.01028973569655742
# LNO-MP2 Orbital Energy: -0.00468166
# LNO-CCSD Orbital Energy: -0.01048118
# LNO-CCSD(T) Orbital Energy: -0.00010332
-0.01048118 - -0.01028973569655742

-0.00019144430344257904

In [9]:
-0.000740/4

-0.000185